In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_hub as hub
classifier = tf.keras.Sequential([
    hub.KerasLayer("https://www.kaggle.com/models/google/mobilenet-v2/frameworks/TensorFlow2/variations/tf2-preview-classification/versions/4", output_shape=[1001])
])
classifier.build([None, 224, 224, 3])

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [2]:
import cv2 as cv
img_data = cv.imread('hen.jpg')
img_data = cv.resize(img_data,(224,224))
img_data = img_data/255
img_data = img_data[None, ...]
# img_data.dtype   => dtype('float64')
wht = classifier.predict(img_data)
wht

1/1 [==============================] - 0s 460ms/step


array([[ 0.36281398,  0.21841888,  0.87137544, ..., -0.60513055,
        -1.6084822 , -1.3960671 ]], dtype=float32)

In [3]:
import numpy as np
np.argmax(wht)
# np.max(wht)

81

In [4]:
# dataset downloaded from "https://www.kaggle.com/datasets/imsparsh/flowers-dataset"
data_dir = "C:/Users/Chinmay/Downloads/tf/dataset"
data_tulip = "C:/Users/Chinmay/Downloads/tf/dataset/tulips"
import pathlib
import os
data_dir = pathlib.Path(data_dir)            # converts image path to windows path
flowers=[]
for i in os.listdir(data_dir):
    flowers.append(i)
print(flowers)

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [5]:

# another way fo making this dictionary
flower_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

    
flower_label = {
    "roses" : 0,
    "daisy" : 1,
    "dandelion" : 2,
    "sunflowers" : 3,
    "tulips" : 4
}
x,y = [],[]

for i in flower_images_dict:
    for img in flower_images_dict[i]:
        f_img = cv.imread(str(img))
        f_img_resized = cv.resize(f_img,(224,224))
        x.append(f_img_resized)
        y.append(flower_label[i])
x = np.array(x)
y = np.array(y)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size= 0.25, random_state=42)
x_train = x_train/255
x_test = x_test/255

In [6]:
# for i in flower_images_dict:
#     print(flower_images_dict[i][0])

In [7]:
# axe = np.array([x_train[0],x_train[1],x_train[2]])

In [8]:
# cv.imshow('img',x_train[0])
# cv.waitKey(0)        # waits infinte time for any keyboard key to press

In [9]:
# predicted = classifier.predict(axe)

In [10]:
# predicted

In [11]:
# predicted_classes=[np.argmax(element) for element in predicted]

In [12]:
# predicted_classes

In [13]:
# names= []
# with open(r"C:\Users\Chinmay\Downloads\tf\ImageNetLabels.txt", 'r') as file:
#     names = file.read().splitlines()
# names

In [14]:
# print(names[986])
# print(names[81])

# transform images into feature vectors

In [15]:
model = tf.keras.Sequential([
    hub.KerasLayer("https://www.kaggle.com/models/google/mobilenet-v2/frameworks/TensorFlow2/variations/tf2-preview-classification/versions/4",input_shape=(224, 224, 3),trainable=False),
    layers.Dense(5, activation='softmax')
])

In [16]:
# model2 = Sequential([
#     hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",trainable=False),
#     layers.Dense(5, activation='softmax')
# ])
# model2.build([None,224, 224, 3])

In [17]:
x_train.shape

(2752, 224, 224, 3)

In [18]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1001)              3540265   
                                                                 
 dense (Dense)               (None, 5)                 5010      
                                                                 
Total params: 3,545,275
Trainable params: 5,010
Non-trainable params: 3,540,265
_________________________________________________________________


In [19]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])
model.fit(x_train,y_train,epochs=5)

Epoch 1/5
86/86 [==============================] - 48s 510ms/step - loss: 0.7863 - accuracy: 0.7140
Epoch 2/5
86/86 [==============================] - 39s 451ms/step - loss: 0.3952 - accuracy: 0.8576
Epoch 3/5
86/86 [==============================] - 38s 447ms/step - loss: 0.3079 - accuracy: 0.8932
Epoch 4/5
86/86 [==============================] - 38s 441ms/step - loss: 0.2652 - accuracy: 0.9062
Epoch 5/5
86/86 [==============================] - 38s 437ms/step - loss: 0.2258 - accuracy: 0.9241


In [20]:
model.evaluate(x_test,y_test)

29/29 [==============================] - 15s 477ms/step - loss: 0.3849 - accuracy: 0.8638


[0.38486477732658386, 0.8638344407081604]

testing accuracy is less than training accuracy

## we can use data augmentation to do better on test dataset

In [21]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomZoom(0.3, input_shape=(224, 224, 3)),
    layers.experimental.preprocessing.RandomContrast(0.3),
    layers.experimental.preprocessing.RandomRotation(0.3)
])

In [22]:
model2 = tf.keras.Sequential([
    data_augmentation,
    hub.KerasLayer("https://www.kaggle.com/models/google/mobilenet-v2/frameworks/TensorFlow2/variations/tf2-preview-classification/versions/4",input_shape=(224, 224, 3),trainable=False),
    layers.Dense(5, activation='softmax')
])


In [24]:
model2.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),metrics=['accuracy'])
model2.fit(x_train,y_train,epochs=10)

Epoch 1/10


86/86 [==============================] - 53s 543ms/step - loss: 0.9895 - accuracy: 0.6592
Epoch 2/10
86/86 [==============================] - 45s 518ms/step - loss: 0.5610 - accuracy: 0.7976
Epoch 3/10
86/86 [==============================] - 44s 510ms/step - loss: 0.4947 - accuracy: 0.8245
Epoch 4/10
86/86 [==============================] - 45s 522ms/step - loss: 0.4714 - accuracy: 0.8376
Epoch 5/10
86/86 [==============================] - 44s 509ms/step - loss: 0.4196 - accuracy: 0.8503
Epoch 6/10
86/86 [==============================] - 44s 509ms/step - loss: 0.4144 - accuracy: 0.8536
Epoch 7/10
86/86 [==============================] - 43s 505ms/step - loss: 0.4093 - accuracy: 0.8605
Epoch 8/10
86/86 [==============================] - 44s 510ms/step - loss: 0.3902 - accuracy: 0.8605
Epoch 9/10
86/86 [==============================] - 44s 508ms/step - loss: 0.3677 - accuracy: 0.8659
Epoch 10/10
86/86 [==============================] - 43s 504ms/step - loss: 0.3711 - accuracy: 0.8692


In [25]:
model.evaluate(x_test,y_test)

29/29 [==============================] - 14s 474ms/step - loss: 0.3849 - accuracy: 0.8638


[0.38486477732658386, 0.8638344407081604]